## Tarea TP10
### Ejercicio 1 Simulacion de horario de atencion de 12pm a 8pm

1.
Realice una  simulación  (de  12pm  a  8am)  considerando  100  réplicas. Para  cada  servicio  indique  cantidad de pacientes que se alcanzaron a atender, tiempo de espera en la cola de ese servicio y tiempo total  que  estuvo trabajando.  Para  todos  las  réplicas  indique  promedio y  desvío  estándar.  Muestre  también  gráficas  de  la evolución  de  las  colas  de  cada servicio  (para  esta  gráfica  utilice  una  única réplica).




# Código principal

In [46]:

#%matplotlib inline
#import matplotlib.pyplot as plt
#from matplotlib.pylab import hist, show

from SimPy.Simulation import *
from SimPy.SimPlot import *   
import random
import numpy as np
import math

#variables para mostrar resultados
resPromedioPacientesD = 0 
resDesvioPacienteD = 0 
resPromColaD = 0
resDesvioColaD = 0 
resPromedioTrabajoD = 0 
resDesvioTrabajoD = 0 
resPromedioPacientesAA = 0 
resDesvioPacienteAA = 0 
resPromColaAA  = 0 
resDesvioColaAA  = 0 
resPromedioTrabajoAA  = 0 
resDesvioTrabajoAA = 0 
resPromedioPacientesRX = 0 
resDesvioPacienteRX = 0 
resPromColaRX  = 0 
resDesvioColaRX = 0 
resPromedioTrabajoRX = 0 
resDesvioTrabajoRX  = 0 
resPromedioPacientesSL = 0 
resDesvioPacienteSL = 0 
resPromColaSL = 0 
resDesvioColaSL = 0 
resPromedioTrabajoSL = 0 
resDesvioTrabajoSL = 0 

# variable global para imprimir en pantalla detalles de la ejecucion
# 0 = no imprimir texto
# 1 = imprimir resultado de los promedio de la ejecucion de todas las replicas
# 2 = imprimir resultado detallado de la ejecucion de todas las réplicas
# 3 = imprimir la actividad de todos los pacientes
debugLevel = 0    
def printDebugDisclaimer():
    print "Valor actual de debugLevel es: ", debugLevel
    print ""
    

## NOTA LA FLECHA DE REVISION INICIAL A DADO DE ALTA NO VA...
class G:
    cantidadPacientesEnColaD = Monitor('Cantidad de Pacientes esperando Diagnostico')   
    cantidadDePacienesAtendidosD = Monitor('Cantidad de Pacientes atendidos Diagnostico')   
    cantidadDePacienesAtendidosAA = Monitor('Cantidad de Pacientes atendidos Atencion Ambulatoria')
    cantidadDePacienesAtendidosRX = Monitor('Cantidad de Pacientes atendidos Rayos X')
    cantidadDePacienesAtendidosSL = Monitor('Cantidad de Pacientes atendidos Servicio Laboratorio')

    tiempoEsperaEnColaD = Monitor('Tiempo de espera en cola Diagnostico')
    tiempoEsperaEnColaAA = Monitor('Tiempo de espera en cola Atencion Ambulatoria')
    tiempoEsperaEnColaRX = Monitor('Tiempo de espera en cola Rayos X')
    tiempoEsperaEnColaSL = Monitor('Tiempo de espera en cola Servicio Laboratorio')

    
    tiempoTrabajoD = Monitor('Tiempo de trabajo Diagnostico')
    tiempoTrabajoAA = Monitor('Tiempo de trabajo Atencion Ambulatoria')
    tiempoTrabajoRX = Monitor('Tiempo de trabajo Rayos X')
    tiempoTrabajoSL = Monitor('Tiempo de trabajo Servicio Laboratorio')

    DIAGNOSTICO = "D"
    ATENCION_AMBULATORIA = "AA"
    RAYOS_X = "RX"
    HOSPITAL = "H"
    SERVICIO_LABORATORIO = "SL"
    ALTA = "A"


class Arrivals(Process):
    # genera arribos aleatorios
    def run(self, N, lamb):
        if(N > 0):
        # genera los arribos de N entidades
            for i in range(N):
                p = Patient(str(i))  # str(i) es el identificador de paciente
                activate(p, p.run())
                # calcula el tiempo del pr�ximo arrivo...
                t = random.expovariate(1. / lamb)
                # ...  y lo planifica para el futuro (tiempo actual de la
                # simulaci�n + t
                yield hold, self, t
        else:
            i = 0
            while(True):    
                p = Patient(str(i))  # str(i) es el identificador de paciente
                activate(p, p.run())
                # calcula el tiempo del pr�ximo arrivo...
                t = random.expovariate(1. / lamb)
                # ...  y lo planifica para el futuro (tiempo actual de la
                # simulaci�n + t
                yield hold, self, t
                i+=1


class Patient(Process):
    global debugLevel
    # se implementa init a los efectos de asignar un identificador a esta
    # instancia de cliente
    def __init__(self, id):
        Process.__init__(self)
        self.id = id
        self.arrivalTime = now()

    def weighted_choice(self,choices):
       total = sum(w for c, w in choices)
       r = random.uniform(0, total)
       upto = 0
       for c, w in choices:
          if upto + w >= r:
             return c
          upto += w
       assert False, "Shouldn't get here"
       
    def run(self):
        if(debugLevel > 2):
            print now(),"Arribo el paciente ",self.id
        #G.cantidadPacientesEnColaD.osbserve(G.DIAGNOSTICO.queue)
        #print "pacientes en cola de diagnostico", G.cantidadPacientesEnColaD.queue()
        yield request,self,G.DIAGNOSTICO        
        G.tiempoEsperaEnColaD.observe(now() - self.arrivalTime)
        typePat = self.weighted_choice([("AA",0.35),("RX",0.2),("H",0.05),("SL",0.45)])
        t = 60 * 0.05
        G.tiempoTrabajoD.observe(t)
        if(debugLevel > 2):
            print now(),"El paciente ",self.id," es diagnosticado ->",typePat 
        yield hold,self,t
        yield release, self, G.DIAGNOSTICO
        if(debugLevel > 2):
            print now(), "Fin Diagnostico del paciente ", self.id
        G.cantidadDePacienesAtendidosD.observe(1)
        

        if (typePat != "H"):

            if(typePat == "AA"):
                arriboAA = now()
                yield request,self,G.ATENCION_AMBULATORIA
                G.tiempoEsperaEnColaAA.observe(now() - arriboAA)
                if(debugLevel > 2):
                    print now(), "El paciente ",self.id," entra en Atencion Ambulatoria"
                t = abs(np.random.normal(60 * 0.25,0.1))
                G.tiempoTrabajoAA.observe(t)
                yield hold,self,t
                yield release, self, G.ATENCION_AMBULATORIA
                if(debugLevel > 2):
                    print now(), "Fin Atencion Ambulatoria  y paciente dado de alta", self.id
                G.cantidadDePacienesAtendidosAA.observe(1)

            elif(typePat == "RX"):
                arriboRX = now()
                yield request,self,G.RAYOS_X
                if(debugLevel > 2):
                    print now(), "El paciente ",self.id," entra en Rayos X"
                G.tiempoEsperaEnColaRX.observe(now() - arriboRX)
                t = abs(np.random.normal(60 * 0.25,0.05))
                G.tiempoTrabajoRX.observe(t)
                yield hold,self,t
                yield release, self, G.RAYOS_X
                if(debugLevel > 2):
                    print now(), "Fin Rayos X ", self.id
                G.cantidadDePacienesAtendidosRX.observe(1)
                typePat = self.weighted_choice([("A",0.6),("H",0.3),("SL",0.1)])
                if (typePat != "H"):
                    if(debugLevel > 2):
                        print now(),"El paciente ",self.id," qda en el Hospital"
                elif(typePat == "SL"):               
                    arriboSL = now()     
                    yield request,self,G.SERVICIO_LABORATORIO
                    G.tiempoEsperaEnColaSL.observe(now() - arriboSL)
                    if(debugLevel > 2):
                        print now(), "El paciente ",self.id," entra en Servicio de Laboratorio"
                    t = abs(np.random.normal(60 * 0.5,0.1))
                    G.tiempoTrabajoSL.observe(t)
                    yield hold,self,t
                    yield release, self, G.SERVICIO_LABORATORIO
                    if(debugLevel > 2):
                        print now(), "Fin Servicio Laboratorio ", self.id
                    G.cantidadDePacienesAtendidosSL.observe(1)
                    typePat = self.weighted_choice([("A",0.9),("H",0.1)])
                    if (typePat != "H"):
                        if(debugLevel > 2):
                            print now(),"El paciente ",self.id," qda en el Hospital"    
                    else:
                        if(debugLevel > 2):
                            print now(),"El paciente ",self.id," qda dado de Alta"
                else:   
                    if(debugLevel > 2):
                        print now(),"El paciente ",self.id," qda dado de Alta"


            else:
                arriboSL = now()
                yield request,self,G.SERVICIO_LABORATORIO
                if(debugLevel > 2):
                    print now(), "El paciente ",self.id," entra en Servicio de Laboratorio"
                G.tiempoEsperaEnColaSL.observe(now() - arriboSL)
                t = abs(np.random.normal(60 * 0.5,0.1))
                G.tiempoTrabajoSL.observe(t)
                yield hold,self,t
                yield release, self, G.SERVICIO_LABORATORIO
                if(debugLevel > 2):
                    print now(), "Fin Servicio Laboratorio ", self.id
                G.cantidadDePacienesAtendidosSL.observe(1)
                typePat = self.weighted_choice([("A",0.9),("H",0.1)])
                if (typePat != "H"):
                    if(debugLevel > 2):
                        print now(),"El paciente ",self.id," qda en el Hospital"
                else:   
                    if(debugLevel > 2):
                        print now(),"El paciente ",self.id," qda dado de Alta"


        else:
            if(debugLevel > 2):
                print now(),"El paciente ",self.id," qda en el Hospital"



promedioPacientesD = Monitor("Promedio cantidad de pacientes Diagnostico")
promedioPacientesAA = Monitor("Promedio cantidad de pacientes Atencion Ambulatoria")
promedioPacientesRX = Monitor("Promedio cantidad de pacientes Rayos X")
promedioPacientesSL = Monitor("Promedio cantidad de pacientes Servicio Laboratorio")

promedioEsperaEnColaD = Monitor("Promedio espera (en minutos) de pacientes en cola Diagnostico")
promedioEsperaEnColaAA = Monitor("Promedio espera (en minutos) de pacientes en cola Ambulatoria")
promedioEsperaEnColaRX = Monitor("Promedio espera (en minutos) de pacientes en cola Rayos X")
promedioEsperaEnColaSL = Monitor("Promedio espera (en minutos) de pacientes en cola Servicio Laboratorio")

promedioTrabajoD = Monitor("Promedio cantidad de pacientes Diagnostico")
promedioTrabajoAA = Monitor("Promedio cantidad de pacientes Atencion Ambulatoria")
promedioTrabajoRX = Monitor("Promedio cantidad de pacientes Rayos X")
promedioTrabajoSL = Monitor("Promedio cantidad de pacientes Servicio Laboratorio")    
    
def model(D,AA,RX,SL, N, lamb, maxtime,replicas):   
    global debugLevel, resPromedioPacientesD, resDesvioPacienteD, resPromColaD, resDesvioColaD, resPromedioTrabajoD, resDesvioTrabajoD, resPromedioPacientesAA, resDesvioPacienteAA, resPromColaAA , resDesvioColaAA , resPromedioTrabajoAA , resDesvioTrabajoAA, resPromedioPacientesRX, resDesvioPacienteRX, resPromColaRX , resDesvioColaRX, resPromedioTrabajoRX, resDesvioTrabajoRX , resPromedioPacientesSL, resDesvioPacienteSL, resPromColaSL, resDesvioColaSL, resPromedioTrabajoSL, resDesvioTrabajoSL
    promedioPacientesD.reset()
    promedioPacientesAA.reset()
    promedioPacientesRX.reset()
    promedioPacientesSL.reset()

    promedioEsperaEnColaD.reset()
    promedioEsperaEnColaAA.reset()
    promedioEsperaEnColaRX.reset()
    promedioEsperaEnColaSL.reset()

    promedioTrabajoD.reset()
    promedioTrabajoAA.reset()
    promedioTrabajoRX.reset()
    promedioTrabajoSL.reset()


    for r in range(replicas):
        # inicializacion del motor de simulacion
        initialize()
        # definimos servers
        G.DIAGNOSTICO = Resource(D)
        G.ATENCION_AMBULATORIA = Resource(AA)
        G.RAYOS_X = Resource(RX)
        G.SERVICIO_LABORATORIO = Resource(SL)
        #reseteo monitores de la replica
        G.cantidadDePacienesAtendidosD.reset()
        G.tiempoEsperaEnColaD.reset()
        G.tiempoTrabajoD.reset()
        G.cantidadDePacienesAtendidosAA.reset()
        G.tiempoEsperaEnColaAA.reset()
        G.tiempoTrabajoAA.reset()
        G.cantidadDePacienesAtendidosRX.reset()
        G.tiempoEsperaEnColaRX.reset()
        G.tiempoTrabajoRX.reset()
        G.cantidadDePacienesAtendidosSL.reset()
        G.tiempoEsperaEnColaSL.reset()
        G.tiempoTrabajoSL.reset()
        
        #  ejecucion
        s = Arrivals()
        activate(s, s.run(N, lamb))
        simulate(until=maxtime)

        promedioPacientesD.observe(G.cantidadDePacienesAtendidosD.count())
        promedioEsperaEnColaD.observe(G.tiempoEsperaEnColaD.total())
        promedioTrabajoD.observe(G.tiempoTrabajoD.total())
        if(debugLevel > 1):
            print ""
            print "**********************************************************"
            print "Diagnostico:"
            print "Cantidad pacientes: ",G.cantidadDePacienesAtendidosD.count()
            print "Tiempo espera en cola (en minutos): ",G.tiempoEsperaEnColaD.total()
            print "Tiempo trabajo total (en minutos): ",G.tiempoTrabajoD.total()

        promedioPacientesAA.observe(G.cantidadDePacienesAtendidosAA.count())
        promedioEsperaEnColaAA.observe(G.tiempoEsperaEnColaAA.total())
        promedioTrabajoAA.observe(G.tiempoTrabajoAA.total())
        if(debugLevel > 1):
            print ""
            print "Atencion Ambulatoria:"
            print "Cantidad pacientes: ",G.cantidadDePacienesAtendidosAA.count()
            print "Tiempo espera en cola (en minutos): ",G.tiempoEsperaEnColaAA.total()
            print "Tiempo trabajo total (en minutos): ",G.tiempoTrabajoD.total()

        promedioPacientesRX.observe(G.cantidadDePacienesAtendidosRX.count())
        promedioEsperaEnColaRX.observe(G.tiempoEsperaEnColaRX.total())
        promedioTrabajoRX.observe(G.tiempoTrabajoRX.total())
        if(debugLevel > 1):
            print ""
            print "Rayos X:"
            print "Cantidad pacientes: ",G.cantidadDePacienesAtendidosRX.count()
            print "Tiempo espera en cola (en minutos): ",G.tiempoEsperaEnColaRX.total()
            print "Tiempo trabajo total (en minutos): ",G.tiempoTrabajoRX.total()

        promedioPacientesSL.observe(G.cantidadDePacienesAtendidosSL.count())
        promedioEsperaEnColaSL.observe(G.tiempoEsperaEnColaSL.total())
        promedioTrabajoSL.observe(G.tiempoTrabajoSL.total())
        if(debugLevel > 1):
            print ""
            print "Servicio Laboratorio:"
            print "Cantidad pacientes: ",G.cantidadDePacienesAtendidosSL.count()
            print "Tiempo espera en cola (en minutos): ",G.tiempoEsperaEnColaSL.total()
            print "Tiempo trabajo total (en minutos): ",G.tiempoTrabajoSL.total()
    
    #resultado de las medias de la ejecucion de todas las replicas
    resPromedioPacientesD = np.around(promedioPacientesD.mean(), decimals=1)
    resDesvioPacienteD = np.around(math.sqrt(promedioPacientesD.var()), decimals=1)
    resPromColaD= np.around(promedioEsperaEnColaD.mean(), decimals=1)
    resDesvioColaD = np.around(math.sqrt(promedioEsperaEnColaD.var()), decimals=1)
    resPromedioTrabajoD = np.around(promedioTrabajoD.mean(), decimals=1)
    resDesvioTrabajoD = np.around(math.sqrt(promedioTrabajoD.var()), decimals=1)

    resPromedioPacientesAA = np.around(promedioPacientesAA.mean(), decimals=1)
    resDesvioPacienteAA = np.around(math.sqrt(promedioPacientesAA.var()), decimals=1)
    resPromColaAA  = np.around(promedioEsperaEnColaAA.mean(), decimals=1)
    resDesvioColaAA  = np.around(math.sqrt(promedioEsperaEnColaAA.var()), decimals=1)
    resPromedioTrabajoAA  = np.around(promedioTrabajoAA.mean(), decimals=1)
    resDesvioTrabajoAA = np.around(math.sqrt(promedioTrabajoAA.var()), decimals=1)

    resPromedioPacientesRX = np.around(promedioPacientesRX.mean(), decimals=1)
    resDesvioPacienteRX = np.around(math.sqrt(promedioPacientesRX.var()), decimals=1)
    resPromColaRX  = np.around(promedioEsperaEnColaRX.mean(), decimals=1)
    resDesvioColaRX = np.around(math.sqrt(promedioEsperaEnColaRX.var()), decimals=1)
    resPromedioTrabajoRX = np.around(promedioTrabajoRX.mean(), decimals=1)
    resDesvioTrabajoRX  = np.around(math.sqrt(promedioTrabajoRX.var()), decimals=1)

    resPromedioPacientesSL = np.around(promedioPacientesSL.mean(), decimals=1)
    resDesvioPacienteSL = np.around(math.sqrt(promedioPacientesSL.var()), decimals=1)
    resPromColaSL = np.around(promedioEsperaEnColaSL.mean(), decimals=1)
    resDesvioColaSL = np.around(math.sqrt(promedioEsperaEnColaSL.var()), decimals=1)
    resPromedioTrabajoSL  = np.around(promedioTrabajoSL.mean(), decimals=1)
    resDesvioTrabajoSL = np.around(math.sqrt(promedioTrabajoSL.var()), decimals=1)
    
    



# Ejecución de simulación
#### Nota de ejecución:

Para imprimir en pantalla el resultado detallado de la ejecucion de todas las réplicas, modificar la asignación de debugLevel a 2 o más

Para imprimir en pantalla la actividad de todos los pacientes, modificar la asignación de debugLevel a 3 o más

In [47]:

# Debug y Disclaimer
debugLevel = 1
printDebugDisclaimer()

#a)
# de 12pm a 8pm y considere 100 replicas..  total 20 horas entonces
#como tamos haciendo en min 20*60=1200 es maxtime, N=0 para q haga while true
#hasta maxtime
replicasCant = 100
maxtimeVal=1200
pacientesHora=4
DVal = 1 # cantidad en servicio de Diagnostico
AAVal = 1 # cantidad en servicio de Atención Ambulatoria
RXVal = 1 # cantidad en servicio de Rayos X
SLVal = 1 # cantidad en Servicio de Laboratorio
NCant = 0 # Cantidad de solicitudes de atención (cero para simular hasta maxtimeVal)

model(D = DVal,AA = AAVal,RX = RXVal,SL = SLVal, N = NCant, lamb = 60 / pacientesHora, maxtime = maxtimeVal, replicas=replicasCant)



Valor actual de debugLevel es:  1



## Métricas de ejecución de la simulación

In [48]:
print ""
print "-----------------------------------------------------------------------"
print "Diagnostico:"
print "Promedio pacientes: ",resPromedioPacientesD
print "Desvio pacientes: ",resDesvioPacienteD
print "Promedio espera en cola (en minutos): ",resPromColaD
print "Desvio espera en cola (en minutos): ",resDesvioColaD
print "Promedio t trabajo (en minutos): ",resPromedioTrabajoD
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoD

print ""
print "Atencion Ambulatoria:"
print "Promedio pacientes: ",resPromedioPacientesAA
print "Desvio pacientes: ",resDesvioPacienteAA
print "Promedio espera en cola (en minutos): ",resPromColaAA
print "Desvio espera en cola (en minutos): ",resDesvioColaAA
print "Promedio t trabajo (en minutos): ",resPromedioTrabajoAA
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoAA

print ""
print "Rayos X:"
print "Promedio pacientes: ",resPromedioPacientesRX
print "Desvio pacientes: ",resDesvioPacienteRX
print "Promedio espera en cola (en minutos): ",resPromColaRX
print "Desvio espera en cola (en minutos): ",resDesvioColaRX
print "Promedio t trabajo (en minutos): ",promedioTrabajoRX.mean()
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoRX

print ""
print "Servicio Laboratorio:"
print "Promedio pacientes: ",resPromedioPacientesSL
print "Desvio pacientes: ",resDesvioPacienteSL
print "Promedio espera en cola (en minutos): ",resPromColaSL
print "Desvio espera en cola (en minutos): ",resDesvioColaSL
print "Promedio t trabajo (en minutos): ",resPromedioTrabajoSL
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoSL



-----------------------------------------------------------------------
Diagnostico:
Promedio pacientes:  79.2
Desvio pacientes:  9.5
Promedio espera en cola (en minutos):  29.2
Desvio espera en cola (en minutos):  12.5
Promedio t trabajo (en minutos):  238.3
Desvio t trabajo (en minutos):  28.4

Atencion Ambulatoria:
Promedio pacientes:  25.5
Desvio pacientes:  5.5
Promedio espera en cola (en minutos):  81.2
Desvio espera en cola (en minutos):  57.0
Promedio t trabajo (en minutos):  387.0
Desvio t trabajo (en minutos):  83.5

Rayos X:
Promedio pacientes:  15.0
Desvio pacientes:  3.7
Promedio espera en cola (en minutos):  25.5
Desvio espera en cola (en minutos):  26.0
Promedio t trabajo (en minutos):  227.984579698
Desvio t trabajo (en minutos):  55.3

Servicio Laboratorio:
Promedio pacientes:  32.0
Desvio pacientes:  4.3
Promedio espera en cola (en minutos):  1631.6
Desvio espera en cola (en minutos):  1191.5
Promedio t trabajo (en minutos):  982.5
Desvio t trabajo (en minutos):  134

In [49]:
#para pruebas de graficos de evolucion de las colas
print G.cantidadDePacienesAtendidosD

[[3.0, 1], [7.058248334152085, 1], [10.058248334152086, 1], [18.4489908099794, 1], [57.500845720406865, 1], [80.64191126687203, 1], [84.29287964794925, 1], [107.49733518868415, 1], [113.14223139214647, 1], [125.52081798749803, 1], [133.14136397949972, 1], [141.18134497809348, 1], [144.18134497809348, 1], [150.22972315735402, 1], [177.24401983005492, 1], [199.31139538329091, 1], [204.7586513467519, 1], [212.50675152326193, 1], [215.958072868116, 1], [219.7450436694826, 1], [224.9094998372648, 1], [228.88633007557172, 1], [281.3585321013485, 1], [284.3585321013485, 1], [312.05527811401925, 1], [315.05527811401925, 1], [336.75039487406656, 1], [342.84671106055555, 1], [353.38865855478656, 1], [358.3712172056977, 1], [363.7742406168402, 1], [432.2219078993274, 1], [436.9527154061198, 1], [443.83896039079707, 1], [452.73752456058304, 1], [462.10945620137755, 1], [477.88643697986026, 1], [480.88643697986026, 1], [502.44323475772757, 1], [505.44323475772757, 1], [523.187653028018, 1], [534.09

### Graficos de evolucion de las colas

In [35]:
plt = SimPlot()              

HistoD = G.tiempoEsperaEnColaD.histogram(low=0.0,high=200.0,nbins=20)  
HistoAA = G.tiempoEsperaEnColaAA.histogram(low=0.0,high=200.0,nbins=20)  
HistoRX = G.tiempoEsperaEnColaRX.histogram(low=0.0,high=200.0,nbins=20)  
HistoSL = G.tiempoEsperaEnColaSL.histogram(low=0.0,high=200.0,nbins=20)  

plt.plotHistogram(HistoD,xlab='Time (min)', title="Espera de Diagnostico", color="red",width=2)  
plt.plotHistogram(HistoAA,xlab='Time (min)', title="Espera de Atencion Ambulatoria", color="black",width=2)  
plt.plotHistogram(HistoRX,xlab='Time (min)', title="Espera de Rayos X", color="blue",width=2)  
plt.plotHistogram(HistoSL,xlab='Time (min)', title="Espera de Servicio de Laboratorio", color="green",width=2)  


plt.mainloop()  #son histogramas que dan valores raros comparados con los promedios

### Comparación ante variación de llegadas de pacientes
La directiva del hospital está interesad a en conocer el efecto que tendría en el sistema un aumento en la llegada de pacientes. En particular se desea analizar el caso de 5 pacientes por hora.   ¿Cómo afectar ía esto al desempeño? Muestre las mismas métricas y gráficas del punto a)  y explique qué sucede en la tasa de cambio del servicio de laboratorio.

In [25]:
debugLevel = 0
printDebugDisclaimer()

#Dado el resultado de la parte a), se compara cambiando la cantidad de pacientes que llegan por hora

#guardo valores originales
promPacientesD = promedioPacientesD.mean()
promPacientesAA = promedioPacientesAA.mean()
promPacientesRX = promedioPacientesRX.mean()
promPacientesSL = promedioPacientesSL.mean()

promEsperaEnColaD=promedioEsperaEnColaD.mean()
promEsperaEnColaAA=promedioEsperaEnColaAA.mean()
promEsperaEnColaRX=promedioEsperaEnColaRX.mean()
promEsperaEnColaSL=promedioEsperaEnColaSL.mean()


promTrabajoD=promedioTrabajoD.mean()
promTrabajoAA=promedioTrabajoAA.mean()
promTrabajoRX=promedioTrabajoRX.mean()
promTrabajoSL=promedioTrabajoSL.mean()
#b)

#nuevo valor de pacientes por hora
nuevoPacientesHora = 5
model(D = DVal,AA = AAVal,RX = RXVal,SL = SLVal, N = NCant, lamb = 60 / nuevoPacientesHora, maxtime = maxtimeVal, replicas=replicasCant)
print ""
print '+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'
print "Diferencia de indicadores comparando 5 arribos de pacientes vs 4 (parte A) "
print ""

difPromPacientesD = promPacientesD - promedioPacientesD.mean()
difPromPacientesAA =promPacientesAA - promedioPacientesAA.mean()
difPromPacientesRX =promPacientesRX - promedioPacientesRX.mean()
difPromPacientesSL =promPacientesSL - promedioPacientesSL.mean()

difPromEsperaEnColaD = promEsperaEnColaD - promedioEsperaEnColaD.mean()
difPromEsperaEnColaAA = promEsperaEnColaAA - promedioEsperaEnColaAA.mean()
difPromEsperaEnColaRX = promEsperaEnColaRX - promedioEsperaEnColaRX.mean()
difPromEsperaEnColaSL = promEsperaEnColaSL - promedioEsperaEnColaSL.mean()

difPromTrabajoD = promTrabajoD - promedioTrabajoD.mean()
difPromTrabajoAA = promTrabajoAA - promedioTrabajoAA.mean()
difPromTrabajoRX = promTrabajoRX - promedioTrabajoRX.mean()
difPromTrabajoSL = promTrabajoSL - promedioTrabajoSL.mean()


perctDifPromPacientesD = np.around(100 * difPromPacientesD/ promedioPacientesD.mean() ,decimals=1)
perctDifPromPacientesAA = np.around(100 * difPromPacientesAA/ promedioPacientesAA.mean() ,decimals=1)
perctDifPromPacientesRX = np.around(100 * difPromPacientesRX/ promedioPacientesRX.mean() ,decimals=1)
perctDifPromPacientesSL = np.around(100 * difPromPacientesSL/ promedioPacientesSL.mean() ,decimals=1)
 
perctDifPromEsperaEnColaD = np.around(100 * difPromEsperaEnColaD/ promedioEsperaEnColaD.mean() ,decimals=1)
perctDifPromEsperaEnColaAA = np.around(100 * difPromEsperaEnColaAA/ promedioEsperaEnColaAA.mean() ,decimals=1)
perctDifPromEsperaEnColaRX = np.around(100 * difPromEsperaEnColaRX/ promedioEsperaEnColaRX.mean() ,decimals=1)
perctDifPromEsperaEnColaSL = np.around(100 * difPromEsperaEnColaSL/ promedioEsperaEnColaSL.mean() ,decimals=1)
 
perctDifPromTrabajoD = np.around(100 * difPromTrabajoD/ promedioTrabajoD.mean() ,decimals=1)
perctDifPromTrabajoAA = np.around(100 * difPromTrabajoAA/ promedioTrabajoAA.mean() ,decimals=1)
perctDifPromTrabajoRX = np.around(100 * difPromTrabajoRX/ promedioTrabajoRX.mean() ,decimals=1)
perctDifPromTrabajoSL = np.around(100 * difPromTrabajoSL/ promedioTrabajoSL.mean() ,decimals=1)



print "pacientes Diagnostico",difPromPacientesD, "(",perctDifPromPacientesD,"%)"
print "pacientes Atencion Ambulatoria",difPromPacientesAA, "(",perctDifPromPacientesAA,"%)"
print "pacientes Rayos X",difPromPacientesRX, "(",perctDifPromPacientesRX,"%)"
print "pacientes Servicio Laboratorio",difPromPacientesSL, "(",perctDifPromPacientesSL,"%)"

print ""
print "Espera (en minutos) Diagnostico",difPromEsperaEnColaD, "(",perctDifPromEsperaEnColaD,"%)"
print "Espera (en minutos) Atencion Ambulatoria",difPromEsperaEnColaAA, "(",perctDifPromEsperaEnColaAA,"%)"
print "Espera (en minutos) Rayos X",difPromEsperaEnColaRX, "(",perctDifPromEsperaEnColaRX,"%)"
print "Espera (en minutos) Servicio Laboratorio",difPromEsperaEnColaSL, "(",perctDifPromEsperaEnColaSL,"%)"

print ""
print "Trabajo (en minutos) Diagnostico",promTrabajoD, "(",perctDifPromTrabajoD,"%)"
print "Trabajo (en minutos) Atencion Ambulatoria",promTrabajoAA, "(",perctDifPromTrabajoAA,"%)"
print "Trabajo (en minutos) Rayos X",promTrabajoRX, "(",perctDifPromTrabajoRX,"%)"
print "Trabajo (en minutos) Servicio Laboratorio",promTrabajoSL, "(",perctDifPromTrabajoSL,"%)"

#conclusion aumenta todo la cantidad de pacientes atendidos por estacion, cantidad de espera en cola, y tiempo de trabajo total para cada estacion 


Valor actual de debugLevel es:  0


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Diferencia de indicadores comparando 5 arribos de pacientes vs 4 (parte A) 

pacientes Diagnostico -21.17 ( -21.1 %)
pacientes Atencion Ambulatoria -7.78 ( -22.8 %)
pacientes Rayos X -3.66 ( -20.0 %)
pacientes Servicio Laboratorio -4.2 ( -11.7 %)

Espera (en minutos) Diagnostico -23.3068700842 ( -45.2 %)
Espera (en minutos) Atencion Ambulatoria -100.204956621 ( -53.4 %)
Espera (en minutos) Rayos X -23.6581224315 ( -50.9 %)
Espera (en minutos) Servicio Laboratorio -1726.46879504 ( -52.9 %)

Trabajo (en minutos) Diagnostico 238.8 ( -21.1 %)
Trabajo (en minutos) Atencion Ambulatoria 399.408324213 ( -22.9 %)
Trabajo (en minutos) Rayos X 222.296324421 ( -19.7 %)
Trabajo (en minutos) Servicio Laboratorio 974.060843307 ( -11.8 %)


### Asegurar tiempo de espera menor a 10 minutos
Considerando el aumento de la tasa de arribos del inciso anterior,  ¿qué medidas sugerir ía implementar para asegurar que en todos los servicios la espera promedio de los pacientes sea menor a 10 minutos? (sólo se puede aumentar la capacidad de los servicios). 

In [30]:
debugLevel = 0
printDebugDisclaimer()
#parte c

DVal = 2
AAVal = 3
RXVal = 2
SLVal = 4
NCant = 0

model(D = DVal,AA = AAVal,RX = RXVal,SL = SLVal, N = NCant, lamb = 60 / 5, maxtime = maxtimeVal, replicas=replicasCant)

print ""
print "-----------------------------------------------------------------------"
print "Diagnostico:"
print "Promedio pacientes: ",resPromedioPacientesD
print "Desvio pacientes: ",resDesvioPacienteD
print "Promedio espera en cola (en minutos): ",resPromColaD
print "Desvio espera en cola (en minutos): ",resDesvioColaD
print "Promedio t trabajo (en minutos): ",resPromedioTrabajoD
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoD

print ""
print "Atencion Ambulatoria:"
print "Promedio pacientes: ",resPromedioPacientesAA
print "Desvio pacientes: ",resDesvioPacienteAA
print "Promedio espera en cola (en minutos): ",resPromColaAA
print "Desvio espera en cola (en minutos): ",resDesvioColaAA
print "Promedio t trabajo (en minutos): ",resPromedioTrabajoAA
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoAA

print ""
print "Rayos X:"
print "Promedio pacientes: ",resPromedioPacientesRX
print "Desvio pacientes: ",resDesvioPacienteRX
print "Promedio espera en cola (en minutos): ",resPromColaRX
print "Desvio espera en cola (en minutos): ",resDesvioColaRX
print "Promedio t trabajo (en minutos): ",promedioTrabajoRX.mean()
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoRX

print ""
print "Servicio Laboratorio:"
print "Promedio pacientes: ",resPromedioPacientesSL
print "Desvio pacientes: ",resDesvioPacienteSL
print "Promedio espera en cola (en minutos): ",resPromColaSL
print "Desvio espera en cola (en minutos): ",resDesvioColaSL
print "Promedio t trabajo (en minutos): ",resPromedioTrabajoSL
print "Desvio t trabajo (en minutos): ",resDesvioTrabajoSL

print ""
print "El valor necesario de las capacidades para que los pacientes esperen menos de 10 minutos en cada servicio son ",DVal ," Revisión Inicial, ", AAVal," Atención Ambulatoria, ",RXVal ," Rayos X "

Valor actual de debugLevel es:  0


-----------------------------------------------------------------------
Diagnostico:
Promedio pacientes:  100.0
Desvio pacientes:  10.7
Promedio espera en cola (en minutos):  3.2
Desvio espera en cola (en minutos):  2.7
Promedio t trabajo (en minutos):  300.9
Desvio t trabajo (en minutos):  32.2

Atencion Ambulatoria:
Promedio pacientes:  32.7
Desvio pacientes:  5.0
Promedio espera en cola (en minutos):  1.3
Desvio espera en cola (en minutos):  3.1
Promedio t trabajo (en minutos):  495.2
Desvio t trabajo (en minutos):  76.8

Rayos X:
Promedio pacientes:  18.8
Desvio pacientes:  4.0
Promedio espera en cola (en minutos):  1.8
Desvio espera en cola (en minutos):  4.7
Promedio t trabajo (en minutos):  288.020773158
Desvio t trabajo (en minutos):  61.4

Servicio Laboratorio:
Promedio pacientes:  41.8
Desvio pacientes:  7.0
Promedio espera en cola (en minutos):  8.0
Desvio espera en cola (en minutos):  14.3
Promedio t trabajo (en minutos):  1286.4
Desvio t

### Valores necesarios para tener un tiempo de respuesta menor a 10 minutos
El valor necesario de las capacidades para que los pacientes esperen menos de 10 minutos en cada servicio son 
* Revisión Inicial: 2
* Atención Ambulatoria: 3
* Rayos X: 2